In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
# import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch import nn
import json
import random
from torch.nn.functional import normalize

In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3, 500),
            nn.ReLU(),
            nn.Linear(500, 500),
            nn.ReLU(),
            nn.Linear(500, 1),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [3]:
with open("./combined-data-with-latency2-ready.json", "r") as f:
    data_file = json.load(f)



data = []
target = []
for d in data_file:
    if "h" in d.keys() and "b" in d.keys() and "c" in d.keys() and "s" in d.keys():
        data.append([d["b"], d["c"], d["s"]])
        target.append(d["h"])

data = torch.Tensor(data)
target = torch.Tensor(target)

In [4]:
shuffle_order = list(range(len(data)))
random.shuffle(shuffle_order)

In [5]:
data = normalize(data, p=1.0)
target = normalize(target, p=1.0, dim=0)

In [6]:
data = [data[i] for i in shuffle_order]
target = [target[i] for i in shuffle_order]

In [7]:
training_data = data[:2000000]
training_target = target[:2000000]
testing_data = data[2000000:]
testing_target = target[2000000:]

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [9]:
model = NeuralNetwork().to(device)

In [10]:
X = torch.rand(3, device=device)
print(X)
logits = model(X)
# pred_probab = nn.Softmax(dim=1)(logits)
# y_pred = pred_probab.argmax(1)
# print(f"Predicted class: {y_pred}")
print(logits)

tensor([0.9143, 0.3286, 0.2369], device='cuda:0')
tensor([0.1030], device='cuda:0', grad_fn=<AddBackward0>)


In [23]:
def train_loop(model, loss_fn, optimizer):
    size = len(data)
    for X, y in data, target:
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 9 == 0:
            # print(f"Batch {batch}/{len(dataloader)} - Loss: {loss.item():>8f}")

def test_loop(data, target, model, loss_fn):
    # print(data)
    # print(target)
    size = len(data)
    # num_batches = len(dataloader)
    correct, bad = 0, 0

    with torch.no_grad():
        for X in range(len(data)):
            y = target[X]
            X = data[X]
            X, y = X.to(device), y.to(device)
            pred = model(X)
            # print(pred)
            # print(y)
            # print()
            if torch.round(pred) == y:
                # print("correct")
                correct += 1
            else:
                # print("Bad")
                bad += 1
            # print()
            # test_loss += loss_fn(pred, y).item()
            # correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # if (len(data) / 10)
    # test_loss /= num_batches
    # .
    # correct /= size
    print(f" Accuracy: {correct/(correct+bad)}%")

In [13]:
epochs = 10
learning_rate = 1e-3

In [26]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1} - ", end="")
    # train_loop(training_data, training_target, model, loss_fn, optimizer)
    test_loop(testing_data[10000:100000], testing_target[:10000], model, loss_fn)
print("Done!")

Epoch 1 - 

IndexError: list index out of range